## Tarea
Tras mostrar opciones para la detección y extracción de información de caras humanas con deepface, la tarea a entregar consiste en proponer dos escenarios de aplicación y desarrollar dos prototipos de temática libre que provoquen reacciones a partir de la información extraída del rostro. Uno de los prototipos deberá incluir el uso de algún modelo entrenado por ustedes para la extracción de información biometríca, similar al ejemplo del género planteado durante la práctica pero con diferente aplicación (emociones, raza, edad...). El otro es de temática completamente libre.

Los detectores proporcionan información del rostro, y de sus elementos faciales. Ideas inmediatas pueden ser filtros, aunque no hay limitaciones en este sentido. La entrega debe venir acompañada de un gif animado o vídeo de un máximo de 30 segundos con momentos seleccionados de las propuestas. Las propuestas se utilizarán para una posterior votación y elección de las mejores entre el grupo. El podio del curso pasado:

In [2]:
import cv2
import time
import FaceNormalizationUtils as faceutils
# My face detectors interface
import FaceDetectors
import numpy as np
from os import walk

c:\Users\antho\anaconda3\envs\VC_P5_1\lib\site-packages\mtcnn\mtcnn.py:34: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
c:\Users\antho\anaconda3\envs\VC_P5_1\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [3]:
class Point(object):
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
        
def mouse_events(event, x, y, flags, params):
    global points_list, accessory_images, accessory_names, image_id, points_images
    if event == cv2.EVENT_LBUTTONUP:
        closer_point = get_closer_point(points_list, x, y)
        if closer_point is not None:
            points_images[closer_point] = accessory_images[accessory_names[image_id]]
    if event == cv2.EVENT_RBUTTONUP:
        closer_point = get_closer_point(points_list, x, y)
        if closer_point is not None and closer_point in points_images:
            del points_images[closer_point]

def get_closer_point(points, x, y):
    min_dist = 10
    min_p = None
    for i in range(len(points)):
        p = points[i]
        dist = np.sqrt((p.x - x)**2 + (p.y - y)**2)
        if dist < min_dist:
            min_dist = dist
            min_p = i
    return min_p

def draw_png(img, frame, point):
    x_start = point.x - img.shape[1] // 2
    y_start = point.y - img.shape[0] // 2
    x_end = x_start + img.shape[1]
    y_end = y_start + img.shape[0]
    
    # Recorta si se sale del frame
    H, W = frame.shape[:2]
    if x_start < 0 or y_start < 0 or x_end > W or y_end > H:
        return  # evita errores, mejor recortar, te puedo dar versión avanzada si quieres

    # Separar canales
    acc_rgb   = img[:, :, :3]
    acc_alpha = img[:, :, 3] / 255.0

    roi = frame[y_start:y_end, x_start:x_end]

    # Hacer blending por cada canal
    for c in range(3):
        roi[:, :, c] = roi[:, :, c] * (1 - acc_alpha) + acc_rgb[:, :, c] * acc_alpha

    frame[y_start:y_end, x_start:x_end] = roi

In [4]:

def filter(frame):
    b, g, r = cv2.split(frame)
    r = np.roll(r, 4, axis=1)
    b = np.roll(b, -4, axis=1)
    frame = cv2.merge((b, g, r))
    return frame

In [ ]:
normalizatorHS = faceutils.Normalization()

# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Fonts
font = cv2.FONT_HERSHEY_SIMPLEX

# Webcam connection
cap = cv2.VideoCapture(0)
# Check for other cameras
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
else:
    print('Camera 0')

    # Face detection and eye model setup
imodoF = 2
imodoE = 1
debug = 0

image_id = 0
points_list = [Point(0,0) for i in range(68)]
points_images = dict()
accessory_images = dict()
show_debug = True

tears_list_right = list()
tear_creation_time_right = list()
tears_list_left = list()
tear_creation_time_left = list()

tear_image = cv2.imread("./Polyphemustears.webp", cv2.IMREAD_UNCHANGED)
tear_image = cv2.resize(tear_image, (32, 32), interpolation=cv2.INTER_AREA)

accessory_names = []
for (dirpath, dirnames, filenames) in walk("./isaac_images"):
    accessory_names.extend(filenames)
    break

accesory_size = 32
accesory_icon_size = 64
use_filter = False

for name in accessory_names:
    img = cv2.imread("./isaac_images/" + name, cv2.IMREAD_UNCHANGED)
    accessory_images[name] = img

#Set camera resolution
cap.set(3,640)
cap.set(4,480)

# Create window
ret, frame = cap.read()
cv2.imshow("Cam", frame)

# Set mouse callback
cv2.setMouseCallback("Cam", mouse_events)

while True:
    # Get frame
    t = time.time()
    ret, frame = cap.read()

    if ret:
        # For HS normalization
        B, G, R = cv2.split(frame)

        # Search face with a specific setup for face and eye detection
        values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[imodoF], FDet.EyeDetectors[imodoE])

        for i in range(len(tears_list_right)):
            if tears_list_right[i] is None:
                continue
            tear = tears_list_right[i]
            creation_time = tear_creation_time_right[i]
            # Simple gravity effect
            elapsed = time.time() - creation_time
            tear.y = tear.y * 1.0 + int(100 * elapsed * elapsed)
            tear.y = int(tear.y)
            if tear.y > frame.shape[0] + 16:
                index = tears_list_right.index(tear)
                tears_list_right[index] = None
                tear_creation_time_right[index] = None
            draw_png(tear_image, frame, tear)

        for i in range(len(tears_list_left)):
            if tears_list_left[i] is None:
                continue
            tear = tears_list_left[i]
            creation_time = tear_creation_time_left[i]
            # Simple gravity effect
            elapsed = time.time() - creation_time
            tear.y = tear.y * 1.0 + int(100 * elapsed * elapsed)
            tear.y = int(tear.y)
            if tear.y > frame.shape[0] + 16:
                index = tears_list_left.index(tear)
                tears_list_left[index] = None
                tear_creation_time_left[index] = None
            draw_png(tear_image, frame, tear)

        if use_filter:
            frame = filter(frame)
        
        if values is not None:
            face, eyes, shape = values

            #draws face container
            [x, y , w, h] = face
            if x > -1:
                if show_debug:
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

                # draws eyes and mask if available
                [lex, ley, rex, rey] = eyes
                if lex > -1:
                    # Show detected facial elements
                    if imodoF > 0:
                        points_list = []
                        for (x, y) in shape:
                            if show_debug:
                                cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)
                            points_list.append(Point(x, y))

                    if show_debug:
                        cv2.circle(frame, ((int)(lex), (int)(ley)), 4, (0, 0, 255), -1)
                        cv2.circle(frame, ((int)(rex), (int)(rey)), 4, (0, 255, 0), -1)
                    
                    # Show accesory selected
                    accessory_name = accessory_names[image_id]
                    accessory_img = accessory_images[accessory_name]


                    image_to_show = cv2.resize(accessory_img, (accesory_icon_size, accesory_icon_size), interpolation=cv2.INTER_AREA)
                    
                    accesory_x = frame.shape[1]*6//7 - image_to_show.shape[1]//2
                    accesory_y = frame.shape[0]*2//4 - image_to_show.shape[0]//2
                    
                    frame[accesory_y:accesory_y+image_to_show.shape[0], accesory_x:accesory_x+image_to_show.shape[1]] = image_to_show[:,:,0:3]

            # draw images in points
            for name, img in points_images.items():
                point = points_list[int(name)]
                referencia = 180 
                ratio = w / referencia
                scaled_size = int(accesory_size * ratio)

                # límites para evitar tamaños inválidos
                scaled_size = max(8, min(scaled_size, 512))

                img_resized = cv2.resize(img, (scaled_size, scaled_size), interpolation=cv2.INTER_AREA)

                draw_png(img_resized, frame, point)
        
        if debug:
            print("Processing time : {:.3f}".format(time.time() - t))

        # Show resulting image
        cv2.putText(frame, FDet.FaceDetectors[imodoF], (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        if imodoF == 1 or imodoF == 2:
            cv2.putText(frame, FDet.EyeDetectors[imodoE], (50, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow('Cam', frame)
        
        # Esc to finish
        tec = cv2.waitKey(40)
        if tec & tec == 27:  # Esc
            break
        # Face detector change
        
        elif tec & 0xFF == ord('d'):
            image_id = image_id + 1
            if image_id >= len(accessory_names):
                image_id = 0
        #Eye detector change
        elif tec & 0xFF == ord('a'):
            image_id = image_id - 1
            if image_id < 0:
                image_id = len(accessory_names) - 1
        elif tec & 0xFF == ord(' '):
            show_debug = not show_debug
        elif tec & 0xFF == ord('s'):
            use_filter = not use_filter
        elif tec & 0xFF == ord('e'):
            tears_list_right.insert(0, Point(points_list[46].x, points_list[46].y + 10))
            tear_creation_time_right.insert(0, time.time())
        elif tec & 0xFF == ord('q'):
            tears_list_left.insert(0, Point(points_list[41].x, points_list[41].y + 10))
            tear_creation_time_left.insert(0, time.time())
        # if tec != -1:
            # print(tec)

# Close windoews and release camera
cap.release()
cv2.destroyAllWindows()

Camera 0
